<a href="https://colab.research.google.com/github/kartiksharma2383/SQL/blob/main/SQL_Advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1163]:
import sqlite3

In [1164]:
import pandas as pd

In [1165]:
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

In [1166]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS EmployeeDemo
(EmployeeID int,
 FirstName varchar(50),
 LastName varchar(50),
 Age int,
 Gender varchar(50)
 )
""")

In [1167]:
df = pd.read_sql_query("SELECT * FROM EmployeeDemo LIMIT 0", conn)
df

,EmployeeID,FirstName,LastName,Age,Gender


In [1168]:
cursor.execute("""
INSERT INTO EmployeeDemo VALUES
(1001, 'Jim', 'Halpert', 26, 'Male'),
(1002, 'Pam', 'Beasley', 30, 'Female'),
(1003, 'Dwight', 'Schrute', 29, 'Male'),
(1004, 'Angela', 'Martin', 31, 'Female'),
(1005, 'Toby', 'Flenderson', 27, 'Female'),
(1006, 'Holly', 'Flax', 28, 'Male')
""")

In [1169]:
df = pd.read_sql_query("SELECT * FROM EmployeeDemo", conn)
df

,EmployeeID,FirstName,LastName,Age,Gender
0,1001,Jim,Halpert,26,Male
1,1002,Pam,Beasley,30,Female
2,1003,Dwight,Schrute,29,Male
3,1004,Angela,Martin,31,Female
4,1005,Toby,Flenderson,27,Female
5,1006,Holly,Flax,28,Male


In [1170]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS EmployeeSalary
(EmployeeID int,
 Job varchar(50),
 EmployeeSalary int
 )
""")

In [1171]:
df = pd.read_sql_query("SELECT * FROM EmployeeSalary LIMIT 0", conn)
df

,EmployeeID,Job,EmployeeSalary


In [1172]:
cursor.execute("""
INSERT INTO EmployeeSalary VALUES
(1001, 'Salesman', '45000'),
(1002, 'Receptionist', '35000'),
(1003, 'Salesman', '65000'),
(1004, 'Accountant', '47000'),
(1005, 'HR', '50000'),
(1006, 'Supplier Relations', '70000'),
(1007, 'Manager', '75000')
""")

In [1173]:
df = pd.read_sql_query("SELECT * FROM EmployeeSalary", conn)
df

,EmployeeID,Job,EmployeeSalary
0,1001,Salesman,45000
1,1002,Receptionist,35000
2,1003,Salesman,65000
3,1004,Accountant,47000
4,1005,HR,50000
5,1006,Supplier Relations,70000
6,1007,Manager,75000


In [1174]:
# CTE(Common Table Expression  used to create temporary, named result sets within a single query. They help make complex queries more readable and organized.
df = pd.read_sql_query("""
WITH CTE_Employee AS(
SELECT FirstName, LastName, Gender,
EmployeeSalary AS Salary,
COUNT(Gender) OVER (PARTITION BY Gender) AS TotalGender,
AVG(EmployeeSalary) OVER (PARTITION BY Gender) AS AvgSal
FROM EmployeeDemo Emp
JOIN EmployeeSalary Sal
ON Emp.EmployeeID = Sal.EmployeeID
WHERE EmployeeSalary > 45000
)
SELECT * FROM CTE_Employee""", conn)
display(df)

,FirstName,LastName,Gender,Salary,TotalGender,AvgSal
0,Angela,Martin,Female,47000,2,48500.0
1,Toby,Flenderson,Female,50000,2,48500.0
2,Dwight,Schrute,Male,65000,2,67500.0
3,Holly,Flax,Male,70000,2,67500.0


In [1175]:
df = pd.read_sql_query("""
WITH CTE_Employee AS(
SELECT FirstName, LastName, Gender,
EmployeeSalary AS Salary,
COUNT(Gender) OVER (PARTITION BY Gender) AS TotalGender,
AVG(EmployeeSalary) OVER (PARTITION BY Gender) AS AvgSal
FROM EmployeeDemo Emp
JOIN EmployeeSalary Sal
ON Emp.EmployeeID = Sal.EmployeeID
WHERE EmployeeSalary > '45000'
)
SELECT FirstName, AvgSal FROM CTE_Employee""", conn)
display(df)

,FirstName,AvgSal
0,Angela,48500.0
1,Toby,48500.0
2,Dwight,67500.0
3,Holly,67500.0


In [1176]:
#Temp Tables(used to break down complex tasks or query and stored intermediate results from EmployeeDemo and EmployeeSalary tables in temporary tables)
cursor.execute("""
CREATE TEMPORARY TABLE temp_Employee(
EmployeeID int,
Job varchar(100),
EmployeeSalary int
)
""")

df = pd.read_sql_query("SELECT * FROM temp_Employee", conn)
df

,EmployeeID,Job,EmployeeSalary


In [1177]:
cursor.execute("""
INSERT INTO temp_Employee VALUES
(1001, 'HR', '45000')
""")

In [1178]:
df = pd.read_sql_query("SELECT * FROM temp_Employee", conn)
df

,EmployeeID,Job,EmployeeSalary
0,1001,HR,45000


In [1179]:
cursor.execute("""
INSERT INTO temp_Employee
SELECT * FROM EmployeeSalary""")

df = pd.read_sql_query("""SELECT * FROM temp_Employee""", conn)
df

,EmployeeID,Job,EmployeeSalary
0,1001,HR,45000
1,1001,Salesman,45000
2,1002,Receptionist,35000
3,1003,Salesman,65000
4,1004,Accountant,47000
5,1005,HR,50000
6,1006,Supplier Relations,70000
7,1007,Manager,75000


In [1180]:
cursor.execute("""
CREATE TEMPORARY TABLE temp_Employee2(
  Job varchar(50),
  EmployeePerJob int,
  AvgAge int,
  AvgSalary int)""")

df = pd.read_sql_query("SELECT * FROM temp_Employee2", conn)
df

,Job,EmployeePerJob,AvgAge,AvgSalary


In [1181]:
cursor.execute("""
INSERT INTO temp_Employee2
SELECT Job, COUNT(Job), AVG(Age), AVG(EmployeeSalary)
FROM EmployeeDemo Emp
JOIN EmployeeSalary Sal
ON Emp.EmployeeID = Sal.EmployeeID GROUP BY Job""")

df = pd.read_sql_query("""SELECT * FROM temp_Employee2""", conn)
df

,Job,EmployeePerJob,AvgAge,AvgSalary
0,Accountant,1,31.0,47000
1,HR,1,27.0,50000
2,Receptionist,1,30.0,35000
3,Salesman,2,27.5,55000
4,Supplier Relations,1,28.0,70000


In [1182]:
cursor.execute("DROP TABLE IF EXISTS temp_Employee2;")
cursor.execute("""
CREATE TEMPORARY TABLE temp_Employee2(
  Job varchar(50),
  EmployeePerJob int,
  AvgAge int,
  AvgSalary int)""")

df = pd.read_sql_query("SELECT * FROM temp_Employee2", conn)
df

cursor.execute("""
INSERT INTO temp_Employee2
SELECT Job, COUNT(Job), AVG(Age), AVG(EmployeeSalary)
FROM EmployeeDemo Emp
JOIN EmployeeSalary Sal
ON Emp.EmployeeID = Sal.EmployeeID
GROUP BY Job""")

df = pd.read_sql_query("""SELECT * FROM temp_Employee2""", conn)
df

,Job,EmployeePerJob,AvgAge,AvgSalary
0,Accountant,1,31.0,47000
1,HR,1,27.0,50000
2,Receptionist,1,30.0,35000
3,Salesman,2,27.5,55000
4,Supplier Relations,1,28.0,70000


In [1183]:
cursor.execute("""
CREATE TABLE EmployeeErrors(
EmployeeID varchar(50),
FirstName varchar(50),
LastName varchar(50)
)
""")

df = pd.read_sql_query("SELECT * FROM EmployeeErrors", conn)
df

cursor.execute("""
INSERT INTO EmployeeErrors Values
('1001', 'Jimbo', 'Halbert'),
('1002', 'Pamela', 'Beasely'),
('1003', 'Toby', 'Flenderson - Fired')""")

df = pd.read_sql_query("SELECT * FROM EmployeeErrors", conn)
df

,EmployeeID,FirstName,LastName
0,1001,Jimbo,Halbert
1,1002,Pamela,Beasely
2,1003,Toby,Flenderson - Fired


In [1184]:
#String functions - TRIM, LTRIM, RTRIM, Replace, Substring, Upper, Lower
#TRIM(used to remove any potential leading or trailing spaces from the EmployeeID column in the EmployeeErrors table)
df = pd.read_sql_query("SELECT EmployeeID, TRIM(EmployeeID) as IDTRIM FROM EmployeeErrors", conn)
df

,EmployeeID,IDTRIM
0,1001,1001
1,1002,1002
2,1003,1003


In [1185]:
#LTRIM(used to remove any spaces at the beginning of the EmployeeID values if they existed)
df = pd.read_sql_query("SELECT EmployeeID, LTRIM(EmployeeID) as IDTRIM FROM EmployeeErrors", conn)
df

,EmployeeID,IDTRIM
0,1001,1001
1,1002,1002
2,1003,1003


In [1186]:
#RTRIM(used to This would remove any spaces at the end of the EmployeeID values if they existed)
df = pd.read_sql_query("SELECT EmployeeID, RTRIM(EmployeeID) as IDTRIM FROM EmployeeErrors", conn)
df

,EmployeeID,IDTRIM
0,1001,1001
1,1002,1002
2,1003,1003


In [1187]:
#REPLACE
df = pd.read_sql_query("""
SELECT LastName, REPLACE(LastName, '- Fired','') as LastNameFixed
FROM EmployeeErrors""", conn)
df

,LastName,LastNameFixed
0,Halbert,Halbert
1,Beasely,Beasely
2,Flenderson - Fired,Flenderson


In [1188]:
#Substring
df = pd.read_sql_query("SELECT SUBSTRING(FirstName, 1, 3) FROM EmployeeErrors", conn)
df

,"SUBSTRING(FirstName, 1, 3)"
0,Jim
1,Pam
2,Tob


In [1189]:
df = pd.read_sql_query("SELECT SUBSTRING(FirstName, 2, 3) FROM EmployeeErrors", conn)
df

,"SUBSTRING(FirstName, 2, 3)"
0,imb
1,ame
2,oby


In [1190]:
df = pd.read_sql_query("SELECT SUBSTRING(LastName, 2, 3) FROM EmployeeErrors", conn)
df

,"SUBSTRING(LastName, 2, 3)"
0,alb
1,eas
2,len


In [1191]:
df = pd.read_sql_query("SELECT SUBSTRING(LastName, 1, 3) FROM EmployeeErrors", conn)
df

,"SUBSTRING(LastName, 1, 3)"
0,Hal
1,Bea
2,Fle


In [1192]:
df = pd.read_sql_query("""
SELECT err.FirstName, dem.FirstName
FROM EmployeeErrors err
JOIN EmployeeDemo dem ON err.FirstName = dem.FirstName""", conn)
df

,FirstName,FirstName
0,Toby,Toby


In [1193]:
df = pd.read_sql_query("""
SELECT SUBSTRING(err.FirstName,1,3), SUBSTRING(dem.FirstName,1,3)
FROM EmployeeErrors err
JOIN EmployeeDemo dem
ON SUBSTRING(err.FirstName,1,3) = SUBSTRING(dem.FirstName,1,3)""", conn)
df

,"SUBSTRING(err.FirstName,1,3)","SUBSTRING(dem.FirstName,1,3)"
0,Jim,Jim
1,Pam,Pam
2,Tob,Tob


In [1194]:
df = pd.read_sql_query("""
SELECT err.FirstName, SUBSTRING(err.FirstName,1,3),
dem.FirstName, SUBSTRING(dem.FirstName,1,3)
FROM EmployeeErrors err
JOIN EmployeeDemo dem
ON SUBSTRING(err.FirstName,1,3) = SUBSTRING(dem.FirstName,1,3)""", conn)
df

,FirstName,"SUBSTRING(err.FirstName,1,3)",FirstName,"SUBSTRING(dem.FirstName,1,3)"
0,Jimbo,Jim,Jim,Jim
1,Pamela,Pam,Pam,Pam
2,Toby,Tob,Toby,Tob


In [1195]:
#Upper and lower substring
df = pd.read_sql_query("SELECT FirstName, LOWER(FirstName) FROM EmployeeErrors", conn)
df

,FirstName,LOWER(FirstName)
0,Jimbo,jimbo
1,Pamela,pamela
2,Toby,toby


In [1196]:
df = pd.read_sql_query("SELECT FirstName, UPPER(FirstName) FROM EmployeeErrors", conn)
df

,FirstName,UPPER(FirstName)
0,Jimbo,JIMBO
1,Pamela,PAMELA
2,Toby,TOBY


In [1197]:
#Subqueries(used to return data that will be used in the main query)
#Subqueries using SELECT
df = pd.read_sql_query("SELECT * FROM EmployeeSalary", conn)

df = pd.read_sql_query("""
SELECT EmployeeID, EmployeeSalary,
(SELECT AVG(EmployeeSalary) FROM EmployeeSalary)
AS AvgSalary FROM EmployeeSalary""", conn)
display(df)

,EmployeeID,EmployeeSalary,AvgSalary
0,1001,45000,55285.714286
1,1002,35000,55285.714286
2,1003,65000,55285.714286
3,1004,47000,55285.714286
4,1005,50000,55285.714286
5,1006,70000,55285.714286
6,1007,75000,55285.714286


In [1198]:
#Subqueries using PARTITON BY or OVER() (OVER() is used for PARTITON BY)
df = pd.read_sql_query("""
SELECT EmployeeID, EmployeeSalary, AVG(EmployeeSalary)
OVER () AS AllAvgEmployeeSalary
FROM EmployeeSalary""", conn)
display(df)

,EmployeeID,EmployeeSalary,AllAvgEmployeeSalary
0,1001,45000,55285.714286
1,1002,35000,55285.714286
2,1003,65000,55285.714286
3,1004,47000,55285.714286
4,1005,50000,55285.714286
5,1006,70000,55285.714286
6,1007,75000,55285.714286


In [1199]:
#Subqueries using FROM
df = pd.read_sql_query("""
SELECT a.EmployeeID, AllAvgEmployeeSalary
FROM (SELECT EmployeeID, EmployeeSalary, AVG(EmployeeSalary)
over() as AllAvgEmployeeSalary
FROM EmployeeSalary) a""", conn)
display(df)

,EmployeeID,AllAvgEmployeeSalary
0,1001,55285.714286
1,1002,55285.714286
2,1003,55285.714286
3,1004,55285.714286
4,1005,55285.714286
5,1006,55285.714286
6,1007,55285.714286


In [1200]:
#Subqueries using WHERE
df = pd.read_sql_query("""
SELECT EmployeeID, Job, EmployeeSalary
FROM EmployeeSalary WHERE EmployeeID in(
  SELECT EmployeeID FROM EmployeeDemo WHERE Age > 30)""", conn)
display(df)

,EmployeeID,Job,EmployeeSalary
0,1004,Accountant,47000


In [1201]:
#Window functions
df = pd.read_sql_query("""
SELECT Gender, AVG(EmployeeSalary) AS Avg_EmployeeSalary
FROM EmployeeDemo dem
JOIN EmployeeSalary sal
ON dem.EmployeeID=sal.EmployeeID
GROUP BY Gender""", conn)
display(df)

,Gender,Avg_EmployeeSalary
0,Female,44000.0
1,Male,60000.0


In [1202]:
df = pd.read_sql_query("""
SELECT Gender, AVG(EmployeeSalary) OVER()
FROM EmployeeDemo dem
JOIN EmployeeSalary sal
ON dem.EmployeeID=sal.EmployeeID
GROUP BY Gender""", conn)
display(df)

,Gender,AVG(EmployeeSalary) OVER()
0,Female,40000.0
1,Male,40000.0


In [1203]:
df = pd.read_sql_query("""
SELECT Gender, AVG(EmployeeSalary) OVER(PARTITION BY Gender)
FROM EmployeeDemo dem
JOIN EmployeeSalary sal
ON dem.EmployeeID=sal.EmployeeID
GROUP BY Gender""", conn)
display(df)

,Gender,AVG(EmployeeSalary) OVER(PARTITION BY Gender)
0,Female,35000.0
1,Male,45000.0


In [1204]:
df = pd.read_sql_query("""
SELECT dem.FirstName, dem.LastName, Gender, AVG(EmployeeSalary) AS Avg_EmployeeSalary
FROM EmployeeDemo dem
JOIN EmployeeSalary sal
ON dem.EmployeeID=sal.EmployeeID
GROUP BY dem.FirstName, dem.LastName, Gender""", conn)
display(df)

,FirstName,LastName,Gender,Avg_EmployeeSalary
0,Angela,Martin,Female,47000.0
1,Dwight,Schrute,Male,65000.0
2,Holly,Flax,Male,70000.0
3,Jim,Halpert,Male,45000.0
4,Pam,Beasley,Female,35000.0
5,Toby,Flenderson,Female,50000.0


In [1205]:
df = pd.read_sql_query("""
SELECT dem.FirstName, dem.LastName, Gender, AVG(EmployeeSalary) OVER(PARTITION BY Gender)
FROM EmployeeDemo dem
JOIN EmployeeSalary sal
ON dem.EmployeeID=sal.EmployeeID
GROUP BY dem.FirstName, dem.LastName, Gender""", conn)
display(df)

,FirstName,LastName,Gender,AVG(EmployeeSalary) OVER(PARTITION BY Gender)
0,Angela,Martin,Female,44000.0
1,Pam,Beasley,Female,44000.0
2,Toby,Flenderson,Female,44000.0
3,Dwight,Schrute,Male,60000.0
4,Holly,Flax,Male,60000.0
5,Jim,Halpert,Male,60000.0


In [1206]:
#Rolling total (it gives output like 45000+35000=80000, 80000+65000=145000 and so on...)
df = pd.read_sql_query("""
SELECT dem.EmployeeID,
dem.FirstName,
dem.LastName,
sal.EmployeeSalary,
SUM(sal.EmployeeSalary)
OVER (ORDER BY dem.EmployeeID)
AS RollingTotalSalary
FROM EmployeeDemo dem
JOIN EmployeeSalary sal
ON dem.EmployeeID = sal.EmployeeID""", conn)
display(df)

,EmployeeID,FirstName,LastName,EmployeeSalary,RollingTotalSalary
0,1001,Jim,Halpert,45000,45000
1,1002,Pam,Beasley,35000,80000
2,1003,Dwight,Schrute,65000,145000
3,1004,Angela,Martin,47000,192000
4,1005,Toby,Flenderson,50000,242000
5,1006,Holly,Flax,70000,312000


In [1207]:
#Triggers(set of SQL statements that are automatically executed (or "triggered") when a specific event occurs in the database.)
cursor.execute("""
CREATE TRIGGER Employee_insert
AFTER INSERT ON EmployeeSalary
BEGIN INSERT INTO EmployeeDemo(EmployeeID, FirstName, LastName)
SELECT NEW.EmployeeID, dem.FirstName, dem.LastName
FROM EmployeeDemo dem
WHERE dem.EmployeeID = NEW.EmployeeID;
END;
""")

cursor.execute("""
INSERT INTO EmployeeSalary(EmployeeID, Job, EmployeeSalary)
VALUES
(1007, 'Manager', 75000),
(1008, 'Intern', 30000);
""")

df = pd.read_sql_query("SELECT EmployeeID, FirstName, LastName, Age, Gender FROM EmployeeDemo", conn)
display(df)

,EmployeeID,FirstName,LastName,Age,Gender
0,1001,Jim,Halpert,26,Male
1,1002,Pam,Beasley,30,Female
2,1003,Dwight,Schrute,29,Male
3,1004,Angela,Martin,31,Female
4,1005,Toby,Flenderson,27,Female
5,1006,Holly,Flax,28,Male


In [1208]:
#Events
